In [1]:
import json

In [2]:
j = '''

{
"a":100,
"b": [1,2,3],
"c": "python",
"d":{
"e":4
,
"f": 5.5}
}
'''

In [5]:
class CustomDecoder(json.JSONDecoder):
    def decode(self,arg):
        print("decode: ", type(arg), arg)
        return "a simple decode"

In [7]:
result = json.loads(j, cls =CustomDecoder )

decode:  <class 'str'> 

{
"a":100,
"b": [1,2,3],
"c": "python",
"d":{
"e":4
,
"f": 5.5}
}



In [8]:
result

'a simple decode'

In [34]:
class Point:
    def __init__(self, x , y):
        self.x = x
        self.y=y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [35]:
j_points = '''
{
"points":[
[10,20],
[-1,-2],
[0.5,0.5]
]

}

'''


j_other = '''
{
"a":1,
"b":2
}
'''

In [36]:
class CustomDecoder(json.JSONDecoder):
    def decode(self,arg):
        if 'points' in arg:
            obj=json.loads(arg)
            return "parsing object for points"
        else:
            return super().decode(arg)

In [37]:
json.loads(j_points, cls=CustomDecoder)

'parsing object for points'

In [38]:
json.loads(j_other, cls =CustomDecoder)

{'a': 1, 'b': 2}

In [39]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj=json.loads(arg)
        if "points" in obj:
            obj['points']=[Point(x,y)
                          for x, y in obj['points']]
        return obj
        

In [40]:
json.loads(j_points, cls = CustomDecoder)

{'points': [Point(x=10, y=20), Point(x=-1, y=-2), Point(x=0.5, y=0.5)]}

In [43]:
json.loads(j_other, cls= CustomDecoder)

{'a': 1, 'b': 2}

In [44]:
json.loads(j, cls = CustomDecoder)

{'a': 100, 'b': [1, 2, 3], 'c': 'python', 'd': {'e': 4, 'f': 5.5}}

In [45]:
import re

In [46]:
pattern = r'"_type"\s*:\s*"point"'

In [48]:
regexp = re.compile(pattern)

In [49]:
print(regexp.search('"a":1'))

None


In [55]:
print(regexp.search('"_type"   :   "point"'))

<re.Match object; span=(0, 21), match='"_type"   :   "point"'>


In [56]:
re.search(pattern,'"_type"   :   "point"' )

<re.Match object; span=(0, 21), match='"_type"   :   "point"'>

In [81]:
class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj=self.make_pts(obj)
        return obj
    
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key ,value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index]=self.make_pts(item)
        return obj

In [82]:
j = '''
{
    "a": 100,
    "b": 0.5,
    "rectangle": {
        "corners": {
            "b_left": {"_type": "point", "x": -1, "y": -1},
            "b_right": {"_type": "point", "x": 1, "y": -1},
            "t_left": {"_type": "point", "x": -1, "y": 1},
            "t_right": {"_type": "point", "x": 1, "y": 1}
        },
        "rotate": {"_type" : "point", "x": 0, "y": 0},
        "interior_pts": [
            {"_type": "point", "x": 0, "y": 0},
            {"_type": "point", "x": 0.5, "y": 0.5}
        ]
    }
}
'''

In [83]:
json.loads(j)

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
   'b_right': {'_type': 'point', 'x': 1, 'y': -1},
   't_left': {'_type': 'point', 'x': -1, 'y': 1},
   't_right': {'_type': 'point', 'x': 1, 'y': 1}},
  'rotate': {'_type': 'point', 'x': 0, 'y': 0},
  'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
   {'_type': 'point', 'x': 0.5, 'y': 0.5}]}}

In [84]:
from pprint import pprint
pprint(json.loads(j, cls=CustomDecoder))

{'a': 100,
 'b': 0.5,
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [85]:
from decimal import Decimal

In [86]:
CustomDecoder = json.JSONDecoder(parse_float=Decimal)

In [87]:
d = CustomDecoder.decode(j)

In [88]:
pprint(d)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': {'_type': 'point', 'x': -1, 'y': -1},
                           'b_right': {'_type': 'point', 'x': 1, 'y': -1},
                           't_left': {'_type': 'point', 'x': -1, 'y': 1},
                           't_right': {'_type': 'point', 'x': 1, 'y': 1}},
               'interior_pts': [{'_type': 'point', 'x': 0, 'y': 0},
                                {'_type': 'point',
                                 'x': Decimal('0.5'),
                                 'y': Decimal('0.5')}],
               'rotate': {'_type': 'point', 'x': 0, 'y': 0}}}


In [91]:
class CustomDecoder(json.JSONDecoder):
    base_decoder = json.JSONDecoder(parse_float=Decimal)
    
    def decode(self, arg, parse_float=Decimal):
        obj=self.base_decoder.decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj=self.make_pts(obj)
        return obj
    
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key ,value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index]=self.make_pts(item)
        return obj

In [92]:
pprint(json.loads(j, cls = CustomDecoder))

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [93]:
result = json.loads(j, cls=CustomDecoder)

In [94]:
pt = result['rectangle']['interior_pts'][1]

In [95]:
type(pt.x)

decimal.Decimal

In [100]:
class CustomDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        super().__init__(parse_float=Decimal)
    
    def decode(self, arg, parse_float=Decimal):
        obj=super().decode(arg)
        pattern = r'"_type"\s*:\s*"point"'
        if re.search(pattern, arg):
            obj=self.make_pts(obj)
        return obj
    
    
    def make_pts(self, obj):
        if isinstance(obj, dict):
            if obj.get('_type', None) == 'point':
                obj = Point(obj['x'], obj['y'])
            else:
                for key ,value in obj.items():
                    obj[key] = self.make_pts(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index]=self.make_pts(item)
        return obj

In [101]:
result = json.loads(j, cls=CustomDecoder)

In [102]:
pprint(result)

{'a': 100,
 'b': Decimal('0.5'),
 'rectangle': {'corners': {'b_left': Point(x=-1, y=-1),
                           'b_right': Point(x=1, y=-1),
                           't_left': Point(x=-1, y=1),
                           't_right': Point(x=1, y=1)},
               'interior_pts': [Point(x=0, y=0), Point(x=0.5, y=0.5)],
               'rotate': Point(x=0, y=0)}}


In [103]:
pt = result['rectangle']['interior_pts'][1]

In [104]:
type(pt.y)

decimal.Decimal